# PyGeM

## Tutorial 3: Free Form Deformation on a unv mesh on cylinder: continuity of the deformation

In this tutorial we show how to perform the Free Form Deformation on a hexaedral mesh of a cylinder. In particular, above what already seen in the previous tutorials, we give some information about how to set the desired continuity to the geometry we are morphing.

First of all we just import pygem package and we read a parameters file.

In [1]:
import pygem as pg

params = pg.params.FFDParameters()
params.read_parameters(filename='../tests/test_datasets/parameters_test_ffd_pipe_unv_C0.prm')

The following is the parameters file for the case at hand. In particular, if you look at the Box info section, there is 2-by-2-by-3 lattice around a cylinder. Since we want to shift the middle section of the cylinder along the x direction we modify only the parameter x weights corresponding to the middle point in z direction (index==1). In the following we show only the important parts of the parameters file

[Box info] 

n control points x: 2  
n control points y: 2  
n control points z: 3  
  
box lenght x: 2.2  
box lenght y: 2.2  
box lenght z: 6.0  

box origin x: -1.1  
box origin y: -1.1  
box origin z:  2.0  
  
  
[Parameters weights]  

parameter x:  

              0   0   1   0.8
              0   1   1   0.8
              1   0   1   0.8
              1   1   1   0.8

We now load the unv file...

In [2]:
unv_handler = pg.unvhandler.UnvHandler()
mesh_points = unv_handler.parse('../tests/test_datasets/test_pipe.unv')

and visualize the undeformed mesh.

![](pictures/cylinder_orig_unv.png)

We now, as always, perform the FFD and write out the results in another unv file.

In [3]:
free_form = pg.freeform.FFD(params, mesh_points)
free_form.perform()
new_mesh_points = free_form.modified_mesh_points

unv_handler.write(new_mesh_points, 'test_pipe_mod_C0.unv')

Let us see the result...

![](pictures/cylinder_mod_unv_C0.png)

As you can easily see the ffd deformation makes the mesh only C0 continuous.

This is not wrong a priori, but it can have some drawbacks.

First of all, if you start with a smooth geometry, probably you do not want to end up with a geometry with more edges.

For large deformations this can cause inaccurancy errors when you try to perform analysis on the deformed mesh. In fact, it can happen that we have very stretched cells that the solver can not treat properly. Of course it depends mainly on the solver choosen, but, in general, it is not very clever to have "spiky" cells.

Moreover, if you are interested in some quantities, such as grandients and normal vectors, it can be difficult to compute them close to the "artificial edges".

Thus, we can rely on the properties of the Bernstein polynomials (on which FFD is built upon) to overcome this potential problem. In fact, if we add 2 new control points in the z direction we can move only the middle one, leaving the first two and the last two still. The new parameter file (only the relevant changes) becomes:

[Box info] 

n control points x: 2  
n control points y: 2  
n control points z: 5  
  
box lenght x: 2.2   
box lenght y: 2.2   
box lenght z: 6.0   

box origin x: -1.1  
box origin y: -1.1  
box origin z:  2.0  


[Parameters weights]  

parameter x: 

              0   0   2   0.8
              0   1   2   0.8
              1   0   2   0.8
              1   1   2   0.8


We again load the new parameter file, perform the FFD and write out the results in another unv file.

In [4]:
params = pg.params.FFDParameters()
params.read_parameters(filename='../tests/test_datasets/parameters_test_ffd_pipe_unv_C1.prm')

free_form = pg.freeform.FFD(params, mesh_points)
free_form.perform()
new_mesh_points = free_form.modified_mesh_points

unv_handler.write(new_mesh_points, 'test_pipe_mod_C1.unv')

And here it is the C1 mesh on the cylinder.

![](pictures/cylinder_mod_unv_C1.png)

You can add some other points to increase again the continuity of the mesh!